# Data Processing

In [31]:
import SpaceScraper
import pandas as pd

### Scrape and create csv

Running the code below will scrape data from https://nextspaceflight.com/ and create a formatted csv file. Process will take ~30 minutes so output has been added as 'nextSpaceFlightData.csv'

In [32]:
# SpaceScraper.create_csv('nextSpaceFlightData.csv')

### Load the data

In [33]:
df = pd.read_csv('nextSpaceFlightData.csv')
df.head()

,Unnamed: 0,mission,time,location,rocket,result,organization,status,price,liftoff thrust,payload to LEO,Payload to GTO,Stages,Strap-ons,Rocket Height,Fairing Diameter,Fairing Height
0,0,Iridium-6 & GRACE-FO,"Tue May 22, 2018 19:47 UTC","SLC-4E, Vandenberg SFB, California, USA",Falcon 9 Block 4,Success,SpaceX,Retired,$62.0 million,"6,804 kN","22,800 kg","8,300 kg",2.0,0.0,70.0 m,5.2 m,13.0 m
1,1,SES-12,"Mon Jun 04, 2018 04:45 UTC","SLC-40, Cape Canaveral SFS, Florida, USA",Falcon 9 Block 4,Success,SpaceX,Retired,$62.0 million,"6,804 kN","22,800 kg","8,300 kg",2.0,0.0,70.0 m,5.2 m,13.0 m
2,2,Test Flight,"Tue Feb 06, 2018 20:45 UTC","LC-39A, Kennedy Space Center, Florida, USA",Falcon Heavy,Success,SpaceX,Active,$97.0 million,"22,819 kN","63,800 kg","26,700 kg",2.0,2.0,70.0 m,5.2 m,13.0 m
3,3,CRS SpX-15,"Fri Jun 29, 2018 09:42 UTC","SLC-40, Cape Canaveral SFS, Florida, USA",Falcon 9 Block 4,Success,SpaceX,Retired,$62.0 million,"6,804 kN","22,800 kg","8,300 kg",2.0,0.0,70.0 m,5.2 m,13.0 m
4,4,Transiting Exoplanet Survey Satellite,"Wed Apr 18, 2018 22:51 UTC","SLC-40, Cape Canaveral SFS, Florida, USA",Falcon 9 Block 4,Success,SpaceX,Retired,$62.0 million,"6,804 kN","22,800 kg","8,300 kg",2.0,0.0,70.0 m,5.2 m,13.0 m


### Preliminary Data Exploration

In [34]:
# get rows and columns
df.shape

(6460, 17)

In [35]:
# get column names
df.columns

Index(['Unnamed: 0', 'mission', 'time', 'location', 'rocket', 'result',
       'organization', 'status', 'price', 'liftoff thrust', 'payload to LEO',
       'Payload to GTO', 'Stages', 'Strap-ons', 'Rocket Height',
       'Fairing Diameter', 'Fairing Height'],
      dtype='object')

In [36]:
# check for duplicate rows
df[df.duplicated()]

,Unnamed: 0,mission,time,location,rocket,result,organization,status,price,liftoff thrust,payload to LEO,Payload to GTO,Stages,Strap-ons,Rocket Height,Fairing Diameter,Fairing Height


In [37]:
# check for total missing rows in each column
df.isna().sum()

Unnamed: 0             0
mission                0
time                   0
location               0
rocket                 0
result                 8
organization           0
status                 0
price               3979
liftoff thrust       714
payload to LEO       859
Payload to GTO      1173
Stages                 1
Strap-ons             60
Rocket Height        587
Fairing Diameter    1381
Fairing Height      1595
dtype: int64

### Data Cleaning

In [38]:
# convert time column to timestamp
df['time']=pd.to_datetime(df['time'])

In [39]:
# create a new column for year
df['year'] = df['time'].map(lambda x: x.year)

In [40]:
# shows aggregate number of missing data points per year
# showsn, lots of missing data points but getting better in last 20 years
years_missing = df.drop('year', axis=1).isna().groupby(df.year, sort=False).sum().reset_index()

with pd.option_context('display.max_rows', None,):
    print(years_missing.set_index('year').sum(axis=1).sort_index())

year
1957      6
1958     76
1959    115
1960    196
1961    284
1962    409
1963    290
1964    394
1965    409
1966    480
1967    390
1968    306
1969    283
1970    256
1971    280
1972    248
1973    215
1974    205
1975    260
1976    224
1977    215
1978    251
1979    133
1980    152
1981    179
1982    152
1983    191
1984    171
1985    146
1986    117
1987    119
1988    132
1989    121
1990    162
1991    134
1992    140
1993    130
1994    160
1995    172
1996    145
1997    178
1998    161
1999    145
2000    137
2001    109
2002    106
2003     87
2004     81
2005     62
2006     57
2007     62
2008     48
2009     63
2010     25
2011     36
2012     23
2013     34
2014     30
2015     30
2016     37
2017     43
2018     47
2019     51
2020     49
2021     94
2022     87
2023     27
dtype: int64


### Missing data

The table above shows number of cells with missing data, aggregated on a yearly basis. As shown, years previous to 2003 have significantly more missing data, comperative to missions 2003 and onwards. I initially thought to remove all missions between 1957 and 2002 and run analysis on datapoints from 2003 onwards but that would leave too small of a subset of the overall dataset. Since all the missing data are from numeric columns, I decided to fill each missing value with the average value of the column, based on year. More details below:

In [41]:
# drop repeat index-value column 'Unnamed: 0'
df.drop(columns=('Unnamed: 0'), inplace=True)

# remove whitespace from category columns
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.strip()

### String to float

The following lines of code turn columns with numeric values into float datatypes

In [42]:
# turn numerical columns with char into ints and floats
import re
def sub(x):
    try:
        an = re.sub('[^0-9.]','',x)
        mil = float(an)*1000000
        return mil
    except:
        return None
    
df['price'] = df['price'].apply(sub)


In [43]:
def sub1(x):
    try:
        an = re.sub('[^0-9]','',x)
        return float(an)
    except:
        return None

df['payload to LEO'] = df['payload to LEO'].apply(sub1)

df['liftoff thrust'] = df['liftoff thrust'].apply(sub1)

df['Payload to GTO'] = df['Payload to GTO'].apply(sub1)

In [44]:
def sub2(x):
    try:
        an = re.sub('[^0-9.]','',x)
        return float(an)
    except:
        return None
    
df['Rocket Height'] = df['Rocket Height'].apply(sub2)
df['Fairing Diameter'] = df['Fairing Diameter'].apply(sub2)
df['Fairing Height'] = df['Fairing Height'].apply(sub2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6460 entries, 0 to 6459
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   mission           6460 non-null   object             
 1   time              6460 non-null   datetime64[ns, UTC]
 2   location          6460 non-null   object             
 3   rocket            6460 non-null   object             
 4   result            6452 non-null   object             
 5   organization      6460 non-null   object             
 6   status            6460 non-null   object             
 7   price             2481 non-null   float64            
 8   liftoff thrust    5746 non-null   float64            
 9   payload to LEO    5601 non-null   float64            
 10  Payload to GTO    5287 non-null   float64            
 11  Stages            6459 non-null   float64            
 12  Strap-ons         6400 non-null   float64            
 13  Roc

### Drop missing data

Once all the columns have been coverted, next is to find the average of each numerical column based on year. I noticed that price data had not been provided between the years of 1957 and 1963 so I decided to remove all rows which fell between those years

In [45]:
df.groupby('year').mean().head(8)

,price,liftoff thrust,payload to LEO,Payload to GTO,Stages,Strap-ons,Rocket Height,Fairing Diameter,Fairing Height
year,,,,,,,,,
1957,NaN,3016.333333,343.333333,0.000000,2.333333,2.666667,27.800000,2.950000,4.000000
1958,NaN,1484.052632,793.045455,0.000000,3.181818,0.909091,26.004545,2.748000,5.728000
1959,NaN,1803.428571,1575.692308,0.000000,3.125000,0.666667,28.938571,2.580000,6.740000
1960,NaN,2486.666667,2320.545455,488.888889,2.341463,1.375000,32.930909,2.580000,6.740000
1961,NaN,2130.000000,1740.250000,488.888889,2.410714,0.826923,31.030000,2.373333,5.906667
1962,NaN,2744.121212,2142.040816,536.160000,2.464286,1.012658,35.184242,2.241818,5.376364
1963,NaN,2737.210526,2295.153846,299.677419,2.666667,1.515625,33.846905,2.270000,5.462500
1964,63230000.0,2860.430769,3136.016129,439.121951,2.600000,1.795699,35.330563,2.365556,5.887222


In [46]:
clean_df = df[df['year']>1963]

There were also 8 rows where the result/outcome of the mission was not provided, so those rows have been removed as well

In [47]:
# drop 8 rows where result is not given
clean_df = clean_df.drop(clean_df[clean_df['result'].isna()==True].index)
clean_df.isna().sum()

mission                0
time                   0
location               0
rocket                 0
result                 0
organization           0
status                 0
price               3679
liftoff thrust       568
payload to LEO       740
Payload to GTO       957
Stages                 1
Strap-ons             45
Rocket Height        452
Fairing Diameter    1156
Fairing Height      1369
year                   0
dtype: int64

In [48]:
avg = clean_df.groupby('year').mean()
avg.head()

,price,liftoff thrust,payload to LEO,Payload to GTO,Stages,Strap-ons,Rocket Height,Fairing Diameter,Fairing Height
year,,,,,,,,,
1964,6.323000e+07,2860.430769,3136.016129,439.121951,2.600000,1.795699,35.330563,2.365556,5.887222
1965,6.323000e+07,3186.917526,3810.148936,543.492537,2.584000,1.713115,36.390417,2.437759,6.032727
1966,5.900000e+07,3482.000000,4234.223301,455.838235,2.592593,1.676471,36.446484,2.410328,6.224727
1967,2.162857e+08,3555.811321,5444.188679,662.094737,2.697842,1.719424,37.776981,2.347093,6.098831
1968,2.792000e+08,3935.261261,7399.685714,619.042553,2.661538,1.807692,40.152613,2.401522,6.661605


At this point, I realised that stages and strap-ons are more categorical datatypes than numerical, since half a stage or half a strap-on would not make sense. With this in mind I elected to remove rows with this missing values for these 2 columns.

In [49]:
clean_df = clean_df.dropna(subset=['Stages', 'Strap-ons'])

### Replace missing data with average

The loop below replaces each null value with the average value of that given year. For example, if there was a row containing data of a mission in 1964 and price data was missing, the replacement value would be 6.323000e+07 since that was the avg price of a mission in 1964 (as shown in 'avg' dataframe above).

In [50]:
for column in clean_df.columns:
    if clean_df[column].isnull().values.any() == True:
        na_index = clean_df.index[clean_df[column].isna()].tolist()
        for row in na_index:
            the_year = clean_df.loc[row,'year']
            clean_df.loc[row,column] = avg.loc[the_year, column]    

clean_df.isna().sum()

mission             0
time                0
location            0
rocket              0
result              0
organization        0
status              0
price               0
liftoff thrust      0
payload to LEO      0
Payload to GTO      0
Stages              0
Strap-ons           0
Rocket Height       0
Fairing Diameter    0
Fairing Height      0
year                0
dtype: int64

In [51]:
clean_df.describe()

,price,liftoff thrust,payload to LEO,Payload to GTO,Stages,Strap-ons,Rocket Height,Fairing Diameter,Fairing Height,year
count,6.107000e+03,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000
mean,1.622906e+08,5325.710447,8417.561919,1978.013016,2.810545,2.123956,45.252049,3.125353,10.064185,1991.376290
std,2.850611e+08,5152.652485,9686.780008,2844.735485,0.742448,2.320855,11.964466,0.911966,4.183693,18.000188
min,2.500000e+06,133.000000,0.000000,0.000000,1.000000,0.000000,9.540000,0.500000,1.000000,1964.000000
25%,2.000000e+07,2962.000000,3300.000000,0.000000,2.000000,0.000000,38.100000,2.580000,7.140000,1976.000000
50%,6.468000e+07,4456.000000,6860.000000,662.094737,3.000000,2.000000,44.630000,3.000000,9.540000,1988.000000
75%,1.259403e+08,5885.000000,9797.000000,3000.000000,3.000000,4.000000,51.380000,3.800000,13.000000,2008.000000
max,3.000000e+09,45307.000000,140000.000000,40000.000000,5.000000,9.000000,110.600000,5.920000,43.720000,2023.000000


### create a csv of cleaned dataset

In [52]:
clean_df.to_csv('cleanLaunchData.csv')